In [1]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
df = pd.read_csv('train.csv')

In [24]:
df.head()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.0,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.0,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.0,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.0,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.0,-7.349849,0,0_0_4


In [3]:
X = df[['bid_price', 'bid_size', 'ask_price','ask_size', 'near_price', 'matched_size']]

In [4]:
Y = df['target']

In [27]:
#Linear Regression Model Imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer

In [6]:
Y = np.array(Y)
X = np.array(X)

In [7]:
Y = Y.reshape(-1,1)

In [8]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [15]:
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X_scaled)
Y_imputed = imputer.fit_transform(Y)

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(X_imputed, Y_imputed, test_size=0.2, random_state=42)

In [17]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [18]:
model = LinearRegression()
model.fit(X_train_scaled, Y_train)

LinearRegression()

In [21]:
Y_pred = model.predict(X_test_scaled)

In [28]:
mse = mean_squared_error(Y_test, Y_pred)
me = mean_absolute_error(Y_test, Y_pred)
print("predicted:", Y_pred)
print("Actual: ", Y_test)
print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {me}')

predicted: [[-0.06967922]
 [ 0.13564423]
 [-0.21681985]
 ...
 [-0.02311417]
 [-0.31368322]
 [-0.60977704]]
Actual:  [[ 4.2700768]
 [-3.61979  ]
 [-2.2500753]
 ...
 [-0.7599592]
 [17.689466 ]
 [-1.3101101]]
Mean Squared Error: 89.3894506236317
Mean Absolute Error: 6.407257669137474
